# ATAC-seq Processing Pipeline - Student Exercise

## Learning Objectives
By completing this notebook, you will:
- Understand the ATAC-seq data analysis workflow
- Learn about chromatin accessibility analysis
- Process paired-end ATAC-seq reads to identify open chromatin regions
- Calculate important quality metrics (PBC1, PBC2, NRF, FRiP)

## Pipeline Overview
```
Raw FASTQ → Trimming → Alignment → Filter → Mark Duplicates → Peak Calling
   (Input)  (Trim Galore) (Bowtie2)  (QC)    (Picard)        (MACS2)
```

## What is ATAC-seq?
**ATAC-seq** (Assay for Transposase-Accessible Chromatin) identifies open chromatin regions where DNA is accessible for transcription factor binding.

## Quality Metrics Explained
- **PBC1** (PCR Bottlenecking Coefficient 1): Library complexity indicator
- **PBC2** (PCR Bottlenecking Coefficient 2): Duplicate rate indicator  
- **NRF** (Non-Redundant Fraction): Unique reads / Total reads
- **FRiP** (Fraction of Reads in Peaks): Signal-to-noise ratio

## Required Tools
- **Trim Galore** (0.6.10): Adapter trimming
- **Bowtie2** (2.4.1): Short read aligner
- **Samtools** (1.7): BAM file processing
- **Picard** (2.23.4): Duplicate marking
- **MACS2** (2.2.7.1): Peak calling
- **Bedtools** (2.29.2): Genomic interval operations
- **deeptools** (3.5.5): Coverage and visualization

---

**Instructions**: Follow the cells below and complete sections marked with `# TODO`

## Step 1: Import Python Libraries

In [ ]:
import os
import subprocess
from pathlib import Path
from datetime import datetime

print("✓ Libraries imported successfully!")

## Step 2: Define Bioinformatics Tool Containers

In [ ]:
# Container images for each tool
trimgalore_container = "quay.io/biocontainers/trim-galore:0.6.10--hdfd78af_1"
bowtie2_container = "quay.io/biocontainers/bowtie2:2.4.1--py38h1c8e9b9_3"
samtools_container = "quay.io/biocontainers/samtools:1.7--2"
picard_container = "quay.io/biocontainers/picard:2.23.4--0"
macs2_container = "quay.io/biocontainers/macs2:2.2.7.1--py39hbf8eff0_5"
bedtools_container = "quay.io/biocontainers/bedtools:2.29.2--hc088bd4_0"
deeptools_container = "quay.io/biocontainers/deeptools:3.5.5--pyhdfd78af_0"

print("✓ Container images defined")

## Step 3: Set Your Pipeline Parameters

**📝 TODO: Update these paths with your actual data!**

In [ ]:
# TODO: Update these paths to match your data location
fastq1 = "/path/to/your/sample_R1.fq.gz"      # Forward reads
fastq2 = "/path/to/your/sample_R2.fq.gz"      # Reverse reads
basename = "my_atac_sample"                    # Sample name
output_dir = "/path/to/output_directory"       # Where to save results

# Reference genome files (ask your instructor for these paths)
genome_index = "/path/to/bowtie2_index"        # Bowtie2 genome index
genome_fasta = "/path/to/genome.fasta"         # Genome FASTA file

# Reference data for quality control
# TODO: Update these paths to your chromosome sizes and blacklist files
CHROMSIZES = "/path/to/hg38.chrom.sizes"       # Chromosome sizes file
BLACKLIST = "/path/to/GRCh38_unified_blacklist.bed"  # ENCODE blacklist regions

# Analysis settings
threads = 2                                     # Number of CPU threads
# TODO: Update this to your accessible directory for Singularity
BIND_DIR = "/path/to/accessible_directory/"    # Directory accessible to Singularity

print(f"Sample name: {basename}")
print(f"Threads: {threads}")
print("⚠ Remember to update the file paths above before running!")

## Step 4: Create Output Directories

In [ ]:
# Create output directory structure
OUTPUT_DIR = os.path.join(os.path.abspath(output_dir), f"{basename}_results")
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Create subdirectories
qc_dir = os.path.join(OUTPUT_DIR, f"{basename}_qc")
picard_dir = os.path.join(OUTPUT_DIR, f"{basename}_picard")
peaks_dir = os.path.join(OUTPUT_DIR, f"{basename}_peaks")

for d in [qc_dir, picard_dir, peaks_dir]:
    Path(d).mkdir(exist_ok=True)

# Initialize log file
log_file = os.path.join(OUTPUT_DIR, f"{basename}_pipeline.log")

print(f"✓ Output directory: {OUTPUT_DIR}")
print(f"✓ QC directory: {qc_dir}")
print(f"✓ Picard directory: {picard_dir}")
print(f"✓ Peaks directory: {peaks_dir}")

## Step 5: Trim Adapters with Trim Galore

**What does this do?** ATAC-seq uses a special hard trimming strategy (50bp from 5' end) to remove transposase sequences.

**📝 TODO: Run this cell and observe the trimming output.**

In [ ]:
print("=" * 60)
print("STEP 1: Hard trimming to 50bp and adapter removal")
print("=" * 60)

# Step 1: Hard trim to 50bp from 5' end
print("\n[1/2] Hard trimming to 50bp...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{trimgalore_container}",
    "trim_galore",
    "--paired",
    "--hardtrim5", "50",  # ATAC-seq specific: trim to 50bp
    "-j", str(threads),
    "--basename", basename,
    "-o", OUTPUT_DIR,
    fastq1, fastq2
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ Hard trimming completed")
    # Rename output files for consistency
    trimmed1 = os.path.join(OUTPUT_DIR, f"{basename}_R1.50bp_5prime.fq.gz")
    trimmed2 = os.path.join(OUTPUT_DIR, f"{basename}_R2.50bp_5prime.fq.gz")
else:
    print(f"✗ Error: {result.stderr}")

# Step 2: Quality and adapter trimming
print("\n[2/2] Quality and adapter trimming...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{trimgalore_container}",
    "trim_galore",
    "--paired",
    "-j", str(threads),
    "--basename", basename,
    "-o", OUTPUT_DIR,
    trimmed1, trimmed2
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ Adapter trimming completed!")
    fastq1_trimmed = os.path.join(OUTPUT_DIR, f"{basename}_val_1.fq.gz")
    fastq2_trimmed = os.path.join(OUTPUT_DIR, f"{basename}_val_2.fq.gz")
    print(f"  Trimmed files ready for alignment")
else:
    print(f"✗ Error: {result.stderr}")
    
# Question: Why does ATAC-seq require hard trimming to 50bp?

## Step 6: Align Reads with Bowtie2

**What does this do?** Maps reads to the reference genome using Bowtie2 (optimized for short reads).

**📝 TODO: Complete the Bowtie2 command.**

In [ ]:
print("=" * 60)
print("STEP 2: Alignment with Bowtie2")
print("=" * 60)

fastq1_trimmed = os.path.join(OUTPUT_DIR, f"{basename}_val_1.fq.gz")
fastq2_trimmed = os.path.join(OUTPUT_DIR, f"{basename}_val_2.fq.gz")
sam_file = os.path.join(OUTPUT_DIR, f"{basename}.sam")

# TODO: Fill in the correct container name
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{bowtie2_container}",  # TODO: Which container?
    "bowtie2",
    "--very-sensitive",     # Use sensitive alignment mode
    "-p", str(threads),
    "-x", genome_index,     # Bowtie2 index
    "-1", fastq1_trimmed,
    "-2", fastq2_trimmed
]

print("Running Bowtie2 alignment (this may take several minutes)...")
with open(sam_file, 'w') as outfile:
    result = subprocess.run(cmd, stdout=outfile, stderr=subprocess.PIPE, text=True)

if result.returncode == 0:
    print("✓ Alignment completed!")
    print(f"  SAM file: {sam_file}")
    print(f"\nAlignment stats:\n{result.stderr}")
else:
    print(f"✗ Error: {result.stderr}")
    
# Question: What does the overall alignment rate tell you about your data quality?

## Step 7: Filter Reads - Remove Mitochondrial and Unwanted Chromosomes

**What does this do?** Removes reads from mitochondria (technical artifact) and keeps only main chromosomes.

**📝 TODO: Complete the filtering steps.**

In [ ]:
print("=" * 60)
print("STEP 3: Filter reads - remove chrM and unwanted chromosomes")
print("=" * 60)

sam_file = os.path.join(OUTPUT_DIR, f"{basename}.sam")
bam_file = os.path.join(OUTPUT_DIR, f"{basename}.bam")
sorted_bam = os.path.join(OUTPUT_DIR, f"{basename}_sorted.bam")
filtered_bam = os.path.join(OUTPUT_DIR, f"{basename}_filtered.bam")

# Keep only main chromosomes
main_chromosomes = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", 
                   "chr8", "chr9", "chr10", "chr11", "chr12", "chr13", "chr14",
                   "chr15", "chr16", "chr17", "chr18", "chr19", "chr20", 
                   "chr21", "chr22", "chrX"]

print("[1/4] Converting SAM to BAM...")
# TODO: Complete the samtools view command
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",  # TODO: Which container?
    "samtools", "view",
    "-h", "-@", str(threads),
    "-O", "bam",
    "-o", bam_file,
    sam_file
]
subprocess.run(cmd, capture_output=False)
print("✓ SAM to BAM conversion done")

print("[2/4] Sorting BAM file...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "sort",
    "-@", str(threads),
    "-o", sorted_bam,
    bam_file
]
subprocess.run(cmd, capture_output=False)
print("✓ Sorting done")

print("[3/4] Indexing BAM file...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "index",
    "-@", str(threads),
    sorted_bam
]
subprocess.run(cmd, capture_output=False)
print("✓ Indexing done")

print("[4/4] Filtering chromosomes...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "view",
    "-h", "-@", str(threads),
    "-O", "bam",
    "-o", filtered_bam,
    sorted_bam
] + main_chromosomes

subprocess.run(cmd, capture_output=False)
print("✓ Chromosome filtering completed!")

# Question: Why do we remove mitochondrial reads in ATAC-seq?

## Step 8: Calculate Mitochondrial Read Fraction (Quality Metric)

**What does this do?** High mitochondrial fraction indicates poor quality ATAC-seq data.

**Good ATAC-seq: <10% mitochondrial reads**

In [ ]:
print("=" * 60)
print("STEP 4: Calculate mitochondrial read fraction")
print("=" * 60)

sorted_bam = os.path.join(OUTPUT_DIR, f"{basename}_sorted.bam")

# Count total reads
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "view", "-c", sorted_bam
]
result = subprocess.run(cmd, capture_output=False, text=True)
total_reads = int(result.stdout.strip())

# Count chrM reads
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "view", "-c", sorted_bam, "chrM"
]
result = subprocess.run(cmd, capture_output=False, text=True)
chrM_reads = int(result.stdout.strip())

# Calculate fraction
chrM_fraction = chrM_reads / total_reads if total_reads > 0 else 0
chrM_percent = chrM_fraction * 100

print(f"\n📊 Mitochondrial Read Statistics:")
print(f"  Total reads: {total_reads:,}")
print(f"  chrM reads: {chrM_reads:,}")
print(f"  chrM fraction: {chrM_fraction:.4f} ({chrM_percent:.2f}%)")

if chrM_percent < 10:
    print("  ✓ PASS: Good quality (<10% chrM)")
elif chrM_percent < 20:
    print("  ⚠ WARNING: Moderate chrM content (10-20%)")
else:
    print("  ✗ FAIL: High chrM content (>20%) - consider sample quality")

# Save to file
chrM_file = os.path.join(qc_dir, f"{basename}_chrM_fraction.txt")
with open(chrM_file, 'w') as f:
    f.write("chrM_reads\tTotal_reads\tchrM_fraction\n")
    f.write(f"{chrM_reads}\t{total_reads}\t{chrM_fraction}\n")

print(f"\n✓ Results saved to: {chrM_file}")

## Step 9: Mark Duplicates with Picard

**What does this do?** Identifies PCR duplicate reads (same start/end position).

**📝 TODO: Run Picard MarkDuplicates.**

In [ ]:
print("=" * 60)
print("STEP 5: Mark duplicate reads")
print("=" * 60)

filtered_bam = os.path.join(OUTPUT_DIR, f"{basename}_filtered.bam")
sorted_filtered_bam = os.path.join(OUTPUT_DIR, f"{basename}_sorted_filtered.bam")
mkdup_bam = os.path.join(OUTPUT_DIR, f"{basename}_mkdup.bam")
dup_metrics = os.path.join(picard_dir, f"{basename}_dup_metrics.txt")

print("[1/2] Sorting filtered BAM...")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "sort",
    "-@", str(threads),
    "-o", sorted_filtered_bam,
    filtered_bam
]
subprocess.run(cmd, capture_output=False)
print("✓ Sorting done")

print("[2/2] Marking duplicates with Picard...")
# TODO: Complete the Picard command
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{picard_container}",  # TODO: Which container?
    "picard", "MarkDuplicates",
    "-I", sorted_filtered_bam,
    "-O", mkdup_bam,
    "-M", dup_metrics,
    "-REMOVE_DUPLICATES", "false",  # Mark but don't remove yet
    "-ASSUME_SORT_ORDER", "coordinate"
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ Duplicate marking completed!")
    print(f"  Marked BAM: {mkdup_bam}")
    print(f"  Metrics: {dup_metrics}")
    
    # Show duplicate rate
    with open(dup_metrics, 'r') as f:
        for line in f:
            if line.startswith("LIBRARY"):
                next_line = next(f)
                fields = next_line.strip().split('\t')
                if len(fields) > 8:
                    dup_rate = float(fields[8])
                    print(f"\n  Duplicate rate: {dup_rate*100:.2f}%")
                break
else:
    print(f"✗ Error: {result.stderr}")

## Step 10: Calculate Library Complexity (PBC1, PBC2, NRF)

**What does this do?** Measures library quality and diversity.

**Quality Thresholds:**
- **NRF** > 0.9: Excellent, > 0.8: Good, < 0.5: Poor
- **PBC1** > 0.9: Excellent, > 0.7: Good, < 0.5: Poor  
- **PBC2** > 10: Excellent, > 3: Good, < 1: Poor

In [ ]:
print("=" * 60)
print("STEP 6: Calculate library complexity metrics")
print("=" * 60)

mkdup_bam = os.path.join(OUTPUT_DIR, f"{basename}_mkdup.bam")
qc_file = os.path.join(qc_dir, f"{basename}_qc.txt")

print("Calculating PBC1, PBC2, and NRF...")
print("(This may take a few minutes)")

# Sort by query name for complexity calculation
name_sorted_bam = os.path.join(OUTPUT_DIR, f"{basename}_nsorted_tmp.bam")
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "sort", "-n",
    "-@", str(threads),
    "-o", name_sorted_bam,
    mkdup_bam
]
subprocess.run(cmd, capture_output=False)

# Calculate metrics using bedtools
cmd = f"""
singularity exec -e --no-home --bind {BIND_DIR}:{BIND_DIR} docker://{bedtools_container} \
bedtools bamtobed -bedpe -i {name_sorted_bam} | \
awk 'BEGIN{{OFS="\\t"}}{{print $1,$2,$4,$6,$9,$10}}' | \
sort | uniq -c | \
awk 'BEGIN{{mt=0;m0=0;m1=0;m2=0}}
     ($1==1){{m1=m1+1}} 
     ($1==2){{m2=m2+1}} 
     {{m0=m0+1}} 
     {{mt=mt+$1}} 
     END{{printf "%d\\t%d\\t%d\\t%d\\t%f\\t%f\\t%f\\n", mt,m0,m1,m2,m0/mt,m1/m0,m1/m2}}'
"""

result = subprocess.run(cmd, shell=True, capture_output=False, text=True)
metrics = result.stdout.strip()

# Parse and display results
values = metrics.split('\t')
if len(values) == 7:
    total_reads = int(values[0])
    distinct_loci = int(values[1])
    one_read_loci = int(values[2])
    two_read_loci = int(values[3])
    NRF = float(values[4])
    PBC1 = float(values[5])
    PBC2 = float(values[6])
    
    print(f"\n📊 Library Complexity Metrics:")
    print(f"  Total reads: {total_reads:,}")
    print(f"  Distinct loci: {distinct_loci:,}")
    print(f"  NRF (Non-Redundant Fraction): {NRF:.4f}")
    print(f"  PBC1 (Bottlenecking Coefficient 1): {PBC1:.4f}")
    print(f"  PBC2 (Bottlenecking Coefficient 2): {PBC2:.4f}")
    
    # Quality assessment
    print(f"\n  Quality Assessment:")
    print(f"    NRF: {'✓ Excellent' if NRF > 0.9 else '✓ Good' if NRF > 0.8 else '⚠ Moderate' if NRF > 0.5 else '✗ Poor'}")
    print(f"    PBC1: {'✓ Excellent' if PBC1 > 0.9 else '✓ Good' if PBC1 > 0.7 else '⚠ Moderate' if PBC1 > 0.5 else '✗ Poor'}")
    print(f"    PBC2: {'✓ Excellent' if PBC2 > 10 else '✓ Good' if PBC2 > 3 else '⚠ Moderate' if PBC2 > 1 else '✗ Poor'}")
    
    # Save to file
    with open(qc_file, 'w') as f:
        f.write("Total_reads\tDistinct_loci\tOne_read_loci\tTwo_read_loci\tNRF\tPBC1\tPBC2\n")
        f.write(metrics + "\n")
    
    print(f"\n✓ Results saved to: {qc_file}")

# Clean up temporary file
os.remove(name_sorted_bam)

## Step 11: Filter Low Quality Reads and Remove Duplicates

**What does this do?** Removes duplicates and low-quality alignments (MAPQ < 10).

**📝 TODO: Apply final filters.**

In [ ]:
print("=" * 60)
print("STEP 7: Filter low quality reads and duplicates")
print("=" * 60)

mkdup_bam = os.path.join(OUTPUT_DIR, f"{basename}_mkdup.bam")
final_bam = os.path.join(OUTPUT_DIR, f"{basename}_final.bam")

print("Filtering reads with MAPQ < 10 and removing duplicates...")

# TODO: Complete the samtools filter command
# Hint: -F 1024 removes duplicates, -q 10 filters by quality
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "view",
    "-@", str(threads),
    "-F", "1024",     # Remove duplicate flag
    "-q", "10",       # Minimum MAPQ quality
    "-b",             # Output BAM
    "-o", final_bam,
    mkdup_bam
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    # Sort and index
    print("Sorting and indexing final BAM...")
    sorted_final = final_bam.replace('.bam', '_sorted.bam')
    cmd = [
        "singularity", "exec", "-e", "--no-home",
        "--bind", f"{BIND_DIR}:{BIND_DIR}",
        f"docker://{samtools_container}",
        "samtools", "sort",
        "-@", str(threads),
        "-o", sorted_final,
        final_bam
    ]
    subprocess.run(cmd, capture_output=False)
    os.replace(sorted_final, final_bam)
    
    # Index
    cmd = [
        "singularity", "exec", "-e", "--no-home",
        "--bind", f"{BIND_DIR}:{BIND_DIR}",
        f"docker://{samtools_container}",
        "samtools", "index",
        "-@", str(threads),
        final_bam
    ]
    subprocess.run(cmd, capture_output=False)
    
    print("✓ Filtering completed!")
    print(f"  Final BAM: {final_bam}")
else:
    print(f"✗ Error: {result.stderr}")

## Step 12: ATAC-seq Specific Read Shifting

**What does this do?** Shifts reads to represent the center of transposon binding (+4bp for forward, -5bp for reverse).

This is **specific to ATAC-seq** and improves peak resolution!

In [ ]:
print("=" * 60)
print("STEP 8: ATAC-seq read shifting")
print("=" * 60)

final_bam = os.path.join(OUTPUT_DIR, f"{basename}_final.bam")
shifted_bam = os.path.join(OUTPUT_DIR, f"{basename}_final_shifted.bam")

print("Shifting reads for ATAC-seq (adjusting for Tn5 binding)...")
print("Forward strand: +4bp, Reverse strand: -5bp")

cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{deeptools_container}",
    "alignmentSieve",
    "--ATACshift",    # Special ATAC-seq shift mode
    "-b", final_bam,
    "-o", shifted_bam
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ Read shifting completed!")
    print(f"  Shifted BAM: {shifted_bam}")
else:
    print(f"✗ Error: {result.stderr}")
    
# Question: Why do we shift reads in ATAC-seq analysis?

## Step 13: Call Peaks with MACS2

**What does this do?** Identifies regions of open chromatin (peaks).

**📝 TODO: Run MACS2 peak calling.**

In [ ]:
print("=" * 60)
print("STEP 9: Peak calling with MACS2")
print("=" * 60)

final_bam = os.path.join(OUTPUT_DIR, f"{basename}_final.bam")

print("Calling peaks with MACS2...")

# TODO: Complete the MACS2 command
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{macs2_container}",  # TODO: Which container?
    "macs2", "callpeak",
    "-f", "BAMPE",           # Paired-end BAM format
    "-g", "hs",              # Human genome size
    "--keep-dup", "all",     # Keep all reads (already filtered)
    "-n", basename,
    "-t", final_bam,
    "--outdir", peaks_dir
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ Peak calling completed!")
    
    # Count peaks
    peak_file = os.path.join(peaks_dir, f"{basename}_peaks.narrowPeak")
    with open(peak_file, 'r') as f:
        num_peaks = sum(1 for line in f)
    
    print(f"\n  📊 Results:")
    print(f"    Peaks identified: {num_peaks:,}")
    print(f"    Peak file: {peak_file}")
    print(f"    Summit file: {os.path.join(peaks_dir, f'{basename}_summits.bed')}")
    
    # Show first few peaks
    print(f"\n  Preview of peaks:")
    with open(peak_file, 'r') as f:
        for i, line in enumerate(f):
            if i < 5:
                fields = line.strip().split('\t')
                print(f"    {fields[0]}:{fields[1]}-{fields[2]} (score: {fields[6]})")
            else:
                break
else:
    print(f"✗ Error: {result.stderr}")
    
# Question: What does the peak score represent?

## Step 14: Calculate FRiP Score (Fraction of Reads in Peaks)

**What does this do?** Measures signal-to-noise ratio.

**Quality Thresholds:**
- **FRiP** > 0.3: Excellent
- **FRiP** > 0.2: Good
- **FRiP** < 0.1: Poor (failed experiment)

In [ ]:
print("=" * 60)
print("STEP 10: Calculate FRiP score")
print("=" * 60)

final_bam = os.path.join(OUTPUT_DIR, f"{basename}_final.bam")
peak_file = os.path.join(peaks_dir, f"{basename}_peaks.narrowPeak")

print("Calculating Fraction of Reads in Peaks (FRiP)...")

# Count total reads
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{samtools_container}",
    "samtools", "view", "-c", final_bam
]
result = subprocess.run(cmd, capture_output=False, text=True)
total_reads = int(result.stdout.strip())

# Count reads in peaks using bedtools
cmd = f"""
singularity exec -e --no-home --bind {BIND_DIR}:{BIND_DIR} docker://{bedtools_container} \
bedtools sort -i {peak_file} | \
singularity exec -e --no-home --bind {BIND_DIR}:{BIND_DIR} docker://{bedtools_container} \
bedtools merge -i stdin | \
singularity exec -e --no-home --bind {BIND_DIR}:{BIND_DIR} docker://{bedtools_container} \
bedtools intersect -u -nonamecheck -a {final_bam} -b stdin -ubam | \
singularity exec -e --no-home --bind {BIND_DIR}:{BIND_DIR} docker://{samtools_container} \
samtools view -c
"""

result = subprocess.run(cmd, shell=True, capture_output=False, text=True)
reads_in_peaks = int(result.stdout.strip())

# Calculate FRiP
FRiP = reads_in_peaks / total_reads if total_reads > 0 else 0

print(f"\n📊 FRiP Score:")
print(f"  Total reads: {total_reads:,}")
print(f"  Reads in peaks: {reads_in_peaks:,}")
print(f"  FRiP score: {FRiP:.4f} ({FRiP*100:.2f}%)")

if FRiP > 0.3:
    print("  ✓ Excellent quality (>30%)")
elif FRiP > 0.2:
    print("  ✓ Good quality (20-30%)")
elif FRiP > 0.1:
    print("  ⚠ Moderate quality (10-20%)")
else:
    print("  ✗ Poor quality (<10%) - experiment may have failed")

# Append to QC file
qc_file = os.path.join(qc_dir, f"{basename}_qc.txt")
with open(qc_file, 'a') as f:
    f.write(f"\nFRiP:\n{FRiP}\n")

print(f"\n✓ FRiP score saved to: {qc_file}")

## Step 15: Create BigWig Coverage Track

**What does this do?** Creates a normalized genome browser track for visualization.

**📝 TODO: Generate coverage track.**

In [ ]:
print("=" * 60)
print("STEP 11: Generate BigWig coverage track")
print("=" * 60)

final_bam = os.path.join(OUTPUT_DIR, f"{basename}_final.bam")
bigwig_file = os.path.join(OUTPUT_DIR, f"{basename}_final_RPKM_Norm_bs10.bw")

print("Creating normalized coverage track (RPKM normalized)...")

cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{deeptools_container}",
    "bamCoverage",
    "-b", final_bam,
    "-o", bigwig_file,
    "-bs", "10",              # 10bp bins
    "--normalizeUsing", "RPKM",
    "-p", str(threads)
]

result = subprocess.run(cmd, capture_output=False, text=True)

if result.returncode == 0:
    print("✓ BigWig file created!")
    print(f"  File: {bigwig_file}")
    print(f"\n  💡 This file can be loaded into genome browsers like:")
    print(f"     - IGV (Integrative Genomics Viewer)")
    print(f"     - UCSC Genome Browser")
else:
    print(f"✗ Error: {result.stderr}")

## Step 16: Remove Blacklisted Regions from Peaks

**What does this do?** Removes peaks in problematic genomic regions (artifacts, repetitive DNA).

**📝 TODO: Filter peaks using the ENCODE blacklist.**

In [ ]:
print("=" * 60)
print("STEP 12: Remove blacklisted regions from peaks")
print("=" * 60)

peak_file = os.path.join(peaks_dir, f"{basename}_peaks.narrowPeak")
summit_file = os.path.join(peaks_dir, f"{basename}_summits.bed")
filtered_peaks = os.path.join(peaks_dir, f"{basename}_peaks_blacklist_filtered.narrowPeak")
filtered_summits = os.path.join(peaks_dir, f"{basename}_summits_blacklist_filtered.bed")

print("Removing peaks overlapping ENCODE blacklist regions...")

# Filter peaks
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{bedtools_container}",
    "bedtools", "subtract",
    "-A",              # Remove entire peak if any overlap
    "-a", peak_file,
    "-b", BLACKLIST
]

result = subprocess.run(cmd, capture_output=False, text=True)
with open(filtered_peaks, 'w') as f:
    f.write(result.stdout)

# Filter summits
cmd = [
    "singularity", "exec", "-e", "--no-home",
    "--bind", f"{BIND_DIR}:{BIND_DIR}",
    f"docker://{bedtools_container}",
    "bedtools", "subtract",
    "-A",
    "-a", summit_file,
    "-b", BLACKLIST
]

result = subprocess.run(cmd, capture_output=False, text=True)
with open(filtered_summits, 'w') as f:
    f.write(result.stdout)

# Count filtered peaks
with open(filtered_peaks, 'r') as f:
    num_filtered_peaks = sum(1 for line in f)

with open(peak_file, 'r') as f:
    num_original_peaks = sum(1 for line in f)

removed_peaks = num_original_peaks - num_filtered_peaks

print(f"\n✓ Blacklist filtering completed!")
print(f"  Original peaks: {num_original_peaks:,}")
print(f"  Filtered peaks: {num_filtered_peaks:,}")
print(f"  Removed peaks: {removed_peaks:,} ({removed_peaks/num_original_peaks*100:.1f}%)")
print(f"\n  Final peak file: {filtered_peaks}")
print(f"  Final summit file: {filtered_summits}")

# Question: Why do we need to remove blacklisted regions?

## Step 17: Pipeline Summary and Quality Assessment

**Congratulations!** 🎉 You've completed the ATAC-seq pipeline!

Let's review all quality metrics to assess your experiment.

In [ ]:
print("=" * 70)
print("ATAC-SEQ PIPELINE COMPLETE!")
print("=" * 70)

print(f"\n✓ Sample processed: {basename}")
print(f"✓ Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n" + "=" * 70)
print("QUALITY METRICS SUMMARY")
print("=" * 70)

# Read chrM fraction
chrM_file = os.path.join(qc_dir, f"{basename}_chrM_fraction.txt")
if os.path.exists(chrM_file):
    with open(chrM_file, 'r') as f:
        next(f)  # Skip header
        line = f.readline().strip().split('\t')
        chrM_frac = float(line[2])
        print(f"\n1. Mitochondrial Reads: {chrM_frac*100:.2f}%")
        print(f"   {'✓ PASS' if chrM_frac < 0.1 else '⚠ WARNING' if chrM_frac < 0.2 else '✗ FAIL'}")

# Read library complexity
qc_file = os.path.join(qc_dir, f"{basename}_qc.txt")
if os.path.exists(qc_file):
    with open(qc_file, 'r') as f:
        header = f.readline()
        if 'NRF' in header:
            values = f.readline().strip().split('\t')
            NRF = float(values[4])
            PBC1 = float(values[5])
            PBC2 = float(values[6])
            
            print(f"\n2. Library Complexity:")
            print(f"   NRF: {NRF:.4f} {'✓ Excellent' if NRF > 0.9 else '✓ Good' if NRF > 0.8 else '⚠ Moderate' if NRF > 0.5 else '✗ Poor'}")
            print(f"   PBC1: {PBC1:.4f} {'✓ Excellent' if PBC1 > 0.9 else '✓ Good' if PBC1 > 0.7 else '⚠ Moderate' if PBC1 > 0.5 else '✗ Poor'}")
            print(f"   PBC2: {PBC2:.4f} {'✓ Excellent' if PBC2 > 10 else '✓ Good' if PBC2 > 3 else '⚠ Moderate' if PBC2 > 1 else '✗ Poor'}")
        
        # Read FRiP
        for line in f:
            if 'FRiP' in line:
                frip_line = f.readline().strip()
                if frip_line:
                    FRiP = float(frip_line)
                    print(f"\n3. Signal-to-Noise (FRiP): {FRiP:.4f} ({FRiP*100:.2f}%)")
                    print(f"   {'✓ Excellent' if FRiP > 0.3 else '✓ Good' if FRiP > 0.2 else '⚠ Moderate' if FRiP > 0.1 else '✗ Poor'}")

# Peak counts
filtered_peaks = os.path.join(peaks_dir, f"{basename}_peaks_blacklist_filtered.narrowPeak")
if os.path.exists(filtered_peaks):
    with open(filtered_peaks, 'r') as f:
        num_peaks = sum(1 for line in f)
    print(f"\n4. Peaks Called: {num_peaks:,}")
    print(f"   {'✓ PASS' if num_peaks > 10000 else '⚠ Low peak count'}")

print("\n" + "=" * 70)
print("OUTPUT FILES")
print("=" * 70)

print(f"\n📁 Main Results Directory: {OUTPUT_DIR}")
print(f"\n  Key Files:")
print(f"  ├─ {basename}_final.bam              ← Final aligned reads")
print(f"  ├─ {basename}_final_RPKM_Norm_bs10.bw ← Coverage track (load in IGV)")
print(f"  ├─ {basename}_peaks/")
print(f"  │  ├─ {basename}_peaks_blacklist_filtered.narrowPeak  ← Final peaks")
print(f"  │  └─ {basename}_summits_blacklist_filtered.bed       ← Peak summits")
print(f"  └─ {basename}_qc/")
print(f"     ├─ {basename}_qc.txt              ← All QC metrics")
print(f"     └─ {basename}_chrM_fraction.txt   ← Mitochondrial fraction")

print("\n" + "=" * 70)
print("NEXT STEPS")
print("=" * 70)

print("\n1. 📊 Visualize your data:")
print("   - Load BigWig file into IGV or UCSC Genome Browser")
print("   - Examine peak locations relative to genes")

print("\n2. 🔍 Downstream analysis:")
print("   - Motif enrichment analysis (HOMER, MEME)")
print("   - Differential accessibility (DiffBind, DESeq2)")
print("   - Footprinting analysis (TOBIAS, HINT)")
print("   - Peak annotation (ChIPseeker, GREAT)")

print("\n3. 📝 Review questions:")
print("   - What percentage of your genome is accessible?")
print("   - Are peaks enriched at promoters or enhancers?")
print("   - What transcription factors might be active?")

print("\n" + "=" * 70)
print("Well done! You've successfully processed ATAC-seq data! 🎉")
print("=" * 70)